In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np

with open('descriptions.txt') as f:
    lines = f.readlines()
    
print lines[0]
print lines[1]

In [ ]:
df = pd.DataFrame(columns=['title', 'description'])

numDescriptions = 0
index = 0
while numDescriptions < 143:
    title = ' '.join(lines[index].strip().split(' ')[1:])
    description = ''
    index = index + 1
    while not lines[index] == '\n':
        description = description + ' ' + lines[index]
        index = index + 1
    index = index + 1
    
    df.loc[numDescriptions] = [title, description.strip()]
    numDescriptions = numDescriptions + 1
    
df.to_pickle('descriptions.pkl')

print df.head()

In [ ]:
# Extract bigram matrix

from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS

vect = CountVectorizer(ngram_range=(1,2), stop_words = ENGLISH_STOP_WORDS)
X = vect.fit_transform(df.description.values)

In [ ]:
# Latent Dirichlet Allocation

from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_topics=5,max_iter = 100)
y = lda.fit_transform(X)

In [ ]:
# Todo:
# 1) Print out what words appear most frequently in each topic
# 2) Scatter plot the documents (in 2-D or 3-D space)
# 3) Gather statistics (e.g. the ratio of student/normal tickets for each topic)
# 4) Try other topic modeling methods (tf-idf, SVD, other flavors of LDA, etc.)

### Print out what words appear most frequently in each topic

In [ ]:
# I copy this code from example on sklearn 
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [ ]:
n_top_words = 50

print_top_words(lda, vect.get_feature_names(), n_top_words)

### Scatter plot the documents (in 2-D or 3-D space)

In [ ]:
## determine which topic for each show
doc_topics = []
num_documents = y.shape[0]
for doc in range(num_documents):
    t = y[doc, :].argmax()
    doc_topics.append([doc, df.title[doc], t, y[doc, t]])

In [ ]:
df_topic = pd.DataFrame(np.array(doc_topics), columns = ['index', 'perf_name', 'topic_num', 'score'])
df_topic.topic_num = df_topic.topic_num.astype(dtype = np.int16)
df_topic.head()

In [ ]:
## plot the distribution of the performance for each topic
import seaborn as sns
topic_range = range(5)
sns.factorplot(x = 'topic_num', data = df_topic, kind = 'count', palette="BuPu", size=6, aspect=1.5, order = topic_range)